# Adversarial Autoencoders

TODO: fix citations
TODO: fix my implementation and add link to it

## Basics
Researchers are trying to use autoencoders to create generative models. In essence this is done by training the autoencoder and then passing in new latent codes, $z$, to the decoder part of the autoencoder to let it reconstruct a new data sample. The problem is that it's hard to produce $z$:s that will decode to plausible outputs, $x$. To solve this, we want to put a prior on $z$. In this work, this is done by using the *adversarial training* technique from \emph{Generative Adversarial Networks}~(GAN)~\cite{GAN} so I'll first quickly summarize this method.

In GANs, we use a pair of models, G and D. G is a generative model, that generates samples of $x$ based on input noise variable, $z$, on which we have a prior. D is a discriminative model that tries to distinguish samples of $x$ coming from the data distribution, $p_{data}(x)$, from those coming from the generative model G, $p_G(x)$. The goal of D is to output high probability for $x \sim p_{data}(x)$ and low probability for $x \sim p_G(x)$. The goal of G is to try to confuse D by generating samples that closely matches $x$ from the training data. This becomes an adversarial min-max training procedure as follows.

$$\min_G \max_D \mathbb{E}_{p_{data}(x)} \left[ log\ D(x) \right] + \mathbb{E}_{p(z)} \left[ log (1 - D(G(z))) \right]$$

So back to Adversarial Autoencoders (AAE)~\cite{AAE}, where the authors use this adversarial training idea to impose a prior, $p(z)$, on the latent code $z$ in the autoencoder. This is done by seeing the encoder, $q(z|x)$, as the generative model G that generates $z$ and then we use this discriminative model D to distinguish between $z \sim q(z)$ from $z \sim p(z)$ where $q(z)$ is the aggregated posterior $q(z) = \int_x q(z|x)p_{data}(x)dx$ and the $p(z)$ is the "true" distribution that we want our generated $z$:s to match. Here as well, the G model is trained to "confuse" D. Figure~\ref{fig:basic_aae} shows the setupt. This results in the following min-max game.

$$\min_G \max_D \mathbb{E}_{p(z)} \left[ log\ D(z) \right] + \mathbb{E}_{q(z)} \left[ log (1 - D(z)) \right]$$

Training this is done by first minimizing the reconstruction error of the autoencoder, $\mathbb{E}_{q(z \vert x)} \left[ -log\ p(x \vert z) \right]$, and then updating the G and D models according to the min-max game, i.e. first updating D to better distinguish true from false samples and then updating G to maximally confuse D.

<img src="figs/aae/adv_ae.png" width="50%" height="50%">

## Variations of basic AAE

### Choices of encoding distribution $q(z|x)$
It seems beneficial and more general to view the encoder and decoder probabilistically as distributions. Thus different choices for the encoding distribution are discussed in the paper.

In general, more sources of stochasticity gives a smoother latent space. But the authors say they got similar likelihoods for holdout images with all these versions.

* Deterministic, $q(z|x)$ is function of x, like in basic autoencoder.
* Gaussian, $z_i \sim \mathcal{N}(\mu_i(x), \sigma_i(x))$. Mean and variance of gaussian computed by the encoder network. Reparametrization trick~\cite{VAE} to be able to backpropagate through sampling.
* Universal approximator posterior TODO

### Using labels
Labels can be used either with the prior or with the decoder. In both cases you can use this to better shape the distribution of the latent code $z$, which means it will be easier to use this as a generative model.

First, labels can be used with the prior. I think the best example from the paper is to use this with a gaussian mixture prior, in this case you can let each label correspond to its own mode in the mixture. This is done by just using the label as a switch when producing the "true" samples from the prior so that samples for some label would only draw "true" samples from the mode of the mixture corresponding to that label. Figure~\ref{fig:aae_variants}~(a) shows this variant.

Second, labels can be used as input to the decoder. By doing this the model is forced to retain all the information independent of the label in the continuously varied latent $z$, like style information. This can be adapted to semi-supervised setting by assuming that the generative process also includes a latent label variable $y$ in addition to $z$. Thus let the encoding distribution output this variable as well (softmax into onehot-ish) and add another adversarial network to match this to a categorical prior for $y$. Training is the same with an added semi-supervised training phase where $q(y|x)$ is updated to minimize cross-entropy loss for the labelled inputs. These are shown in Figure~\ref{fig:aae_variants}~(b),~(c).

TODO: show figures for these in subfig fig
<img src="figs/aae/adv_ae_semi.png" width="40%" height="40%">
<img src="figs/aae/adv_ae_style.png" width="40%" height="40%">
<img src="figs/aae/adv_ae_style_label.png" width="40%" height="40%">

## Discussion and Thoughts
not sure how the integrals are being computed? are we just sampling for them

If really high dimensional z, will we get a lot of "holes" in this case?

One advantage: dont need to know functional form of prior, just need to be able to get samples from it. Could be useful.

There seems to be some doubts regarding the "validity" of these adversarially trained stuff. They don't really maximize any likelihood and basically just generates pretty samples for some data (like mnist) but I think it remains to be seen whether it actually works well for many types of data. Perhaps these concerns depend on whether you prefer an approach from theory to implementation rather than implementing this and figuring out the theory later since atm it's not really clear why GANs work (?) other than maybe Nash equilibrium (but maybe that's the full theory motivation).

" GANs don't really work today. Practically, they don't really converge, the results are highly cherry-picked with the training stopped when samples look good. If you change a hyperparameter it all falls apart. Fundamentally, it's minimising a lower bound which is dubious." - Reddit comment fhuszar

# References

<mark> <b>The bib file biblio.bib was not found

</b> </mark>(<a id="cit-GAN" href="#call-GAN">?</a>) !! _This reference was not found in biblio.bib _ !!

(<a id="cit-AAE" href="#call-AAE">?</a>) !! _This reference was not found in biblio.bib _ !!

(<a id="cit-VAE" href="#call-VAE">?</a>) !! _This reference was not found in biblio.bib _ !!

